# (Brief Introduction To) Garbage Collection

The goal of this lecture is to provide a brief introduction to garbage collection in the Lettuce language in the context of the language with explicit and implicit references.

## What is Garbage Collection?

We saw the introduction of mutables in the language either in the form of explicit references through the  constructs

~~~
newref(expr)

assignref(expr, expr)

deref(expr)
~~~

or implicit references (akin to scala vars) through the constructs

~~~
let var identifier = expr in expr

assignvar(identifier, expr)
~~~

However, we left out an explicit delete operation in the language with explicit references. In the language with implicit references, we see that `let var` binding creates a new cell in the memory but we cannot delete cells in the memory. This means that if a program creates a lot of cells, it may even run out of memory.

### Example 1

```ocaml
let x = ( let var y = 25 in  (* new cell created in store for y *)
             let z = y + 25 in 
                y + z
         ) in  (* y goes out of scope here but cell in the store persists *)
   let var w = 25 in 
      w + x
```

This is equivalent to the scala program

```scala
{ 
 val x = { 
        var y = 25
        val z = y + 25
        y + z
      }
 var w = 25
 w + x
}
```

In the program above, the let binding for `var y` creates a new cell in the memory that is bound to `y`. But as soon as `y` goes out of scope there is no reference to the cell from the environment. 

---

> In other words, the cell in the store that `y` used to refer to is no longer accessible by the program, once `y` goes out of scope.

---

### Example 2

```ocaml
let rec fibo = function (x) 
                 if (x <= 2)
                 then 1 
                 else (
                   let var z = fibo(x-1) in (* z created here *)
                   let dummy = assignvar(z, z + fibo(x -2)) in 
                     z
                 ) (* z  out of scope here *)
      in 
        fibo(20)
```

Note that each call to fibonacci creates a new cell in the memory for z that is then used to store intermediate results and finally returned. This way `fibonacci(20)` will create roughly 
$1.6^{20}$ cells in the memory. However, once a call to `fibonacci` is finished and `z` goes out of scope. 

> Nevertheless, our current memory allocator does not reclaim a cell that is no longer accessible from the program. 

Because of the large number of calls to `fibonacci`, with each call creating a version of `z`, this program can consume exponentially many memory cells, most of which are no longer accessible from the environment.

## What is Garbage? 

Garbage Collection (GC) refers to identifying and reclaiming memory cells in the store that are no longer required by the program. These will refer to memory cells that are not "reachable" starting from some "in-scope" identifier in the environment. To this end, let us run through some examples to identify what is "reachable" and what is not.

## Example 1

```ocaml
let var x = 10 in 
  let var y = ( let var z = 15 in  
                  let dummy =   assignvar(z, x - 5) in 
                    z
               ) in 
          y
```

_Roughly_ equivalent scala program: 
```scala
{ 
  var x = 10
  var y = { var z = 15
            z = x - 5 // the val dummy is not needed here
            z }
  y
}
```

Let us trace the execution of this program by listing the environments created and the cells in the store.

  1. __Store__ Cell 0 created with value 10 stored in it. __Environment:__ `x` maps to `Ref(0)`
  2. __Store__ Cell 1 created with value 15 in it. __Environment:__ `z` maps to `Ref(1)`
  3. __Store__ Cell 1 updated to value 5. __Environment__: `dummy` mapped to number `NumValue(5)`.
  4.                                      __Environment__: `z` goes out of scope.
  5. __Store__ Cell 2 created with value 5. __Enviroment__: `y` bound to `Ref(2)`.
  
When the program is evaluating the expression `Ident("y")` at the very last line, we see that: 
  - `Cell 0` and `Cell 2` in store are "reachable" from evironment through identifiers `x` and `y` respectively.
  - `Cell 1` used to be reachable from environment through identifier `z` but it went out of scope and `Cell 1` is no longer reachable at this point.

  
__Syntactic vs. Semantic "Garbage"__

In the example, note that `Cell 0` is reached from the environment using the identifier `x` and potentially it is reachable. 
However, note also, that the value of the `x` is never used after the subexpression `assignvar(z, x - 5)` in line 3 has been evaluated. Therefore, it can be regarded as garbage much earlier on and `Cell 0` can be released from the memory early. However, doing so requires us to know at that point in execution that `x` is never going to be used ever again in the future of the evaluation. This is a hard problem.

However, we now distinguish two types of Garbage: 

> __Syntactic Garbage:__ A cell is no longer reachable because every identifier that could reach it, has gone out of scope.

> __Semantic Garbage:__ A cell is no longer reachable because every identifier that can reach it either has gone out of scope or is simply never referred to by the program evaluation going forward.
  
Obviously, the set of syntactic garbage is a subset of the set of semantic garbage.



## Example 2: Indirect Reachability

We will now differentiate between direct and indirect reachability. Let us take an example using the explicit language.

```ocaml
let chain = (let x = newref(10) in 
               let y = newref(x) in 
                  let z = newref(y) in
                     z
            ) in  
      deref(deref(deref(chain))) + 20
```

Here is a picture of how the environment and store looks like when the subexression
`deref(deref(deref(chain)))` is just about to be evaluated.

![Environment and Store For Example 2](figures/figExample2.png)

Notice that `Cell 2` is directly reachable from the environment identifier `chain`. However, cells 0 and 1 are indirectly reachable through cell 2. None of these cells are garbage (either syntactic or semantic). Therefore, it is important to maintain information that is indirect.

## Example 3: Reachability through Closures

Consider the example in the implicit language: 

```ocaml
let foo = ( let var counter = 0 in  (* Create a counter *)
             function (x) {
               let dummy = assignvar(counter, counter + 1) in (* increment counter  *)
                 x + counter  (* add counter to the argument *)
             } 
          ) in (* counter goes out of scope here?? *)
let var y = 5 in 
   foo(y)
```

Equivalent scala program:

```scala
{
  val foo: Int => Int = 
        {
            var counter = 0
            (x: Int) => {
                 counter = counter + 1
                 x + counter
            }
        }

  var y = 5
  foo(y)
}
```


The program creates a var `counter` initialized to `0` and a function `foo` that when called
  - increments counter and
  - adds it to the argument as its return value.
Then it creates a var called `y` initialized to `5`. 

Let `Cell 0` be the first cell created in the store. This cell is initialized to the value 0 and bound to counter in the environment. However, after the let binding for `foo`, 
the var `counter` goes out of scope. Nevertheless, the cell 0 is still reachable? How?

Figure below illustrates how we can reach cell 0 when the function call `foo(y)` is being evaluated: 

![Illustration of reachability through closure capture](figures/figExample3.png)

Therefore, reachability through closures has to be considered.


## Garbage Collection: Adding Extra Information to the Store

As a first step towards a garbage collecting implementation in Lettuce, let us first reimagine how the store works. Thus, far the store has been imagined simply as an array of values. We will first reimagine a store as an array with extra information.

Each cell in the store is a tuple of a value and a integer `refCount` that can be set to a positive value if the store is really reachable or 0 if it is unreachable. Essentially
`refCount` is a "reference counter" that counts how many references exist to the store in the environment of the program.

A cell that has reference count of 0 can be overwritten when the next allocation is requested.

### Example (Store with ref count)


| Address  | Value   | refCount |
|----------|---------|------|
|   0      | NumValue(45) | 2 |
|   1   | Closure(...)  | <font color="red"> 0 </font> | 
|   2   | Ref(0)  | 1 |
|   3   | BoolValue(false)  | 1 |
|   4   | NumValue(40)   | <font color="red"> 0 </font> | 

 
In the example above, we see that certain cells have positive reference counts meaning that they have one or more references from the current environment and some cells have 0 reference count, meaning that they are __definitely not__ reachable from the current environment. A cell with reference count 0 can be recycled/reused. For instance, if a new allocation request arrives, we may "reuse" cell 1 or cell 4 rather than create a new cell 5 in the store.


In [11]:
sealed trait Value
sealed trait Environment
sealed trait Expr
type Address = Int
type RefCount = Int

/*-- Let us define our values so far --*/
case object ErrorValue extends Value
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value 
case class Closure(id: String, e: Expr, env: Environment) extends Value
case class Reference(j: Address) extends Value

/*-- Let us define the environment --*/
case object EmptyEnvironment extends Environment
case class Extend(x: String, v: Value, env: Environment) extends Environment
case class ExtendRec(f: String, x: String, e: Expr, env: Environment) extends Environment

def lookupEnv(x: String, env: Environment):Value = env match {
    case EmptyEnvironment => throw new IllegalArgumentException(s"could not find identifier $x")
    case Extend(x1, v1, _) if (x == x1) => v1
    case Extend(_, _, env1) => lookupEnv(x, env1)
    case ExtendRec(f, param, e, _) if (x == f) => Closure(param, e, env)
    case ExtendRec(_, _, _, env1 ) => lookupEnv(x, env1)
}

/*-- Let us define Expressions --*/

/* Basics */
case class Const(f: Double) extends Expr
case class ConstBool(b: Boolean) extends Expr
case class Ident(x: String) extends Expr

/* Arithmetic Booleans If then Else etc.. */
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr

/* Let Binding */
case class Let(x: String, e1: Expr, e2: Expr ) extends Expr

/* Functions and recursive functions */
case class Fundef(x: String, body: Expr) extends Expr
case class Funcall(e1: Expr, e2: Expr) extends Expr 
case class LetRec(f: String, x: String, e1: Expr, e2: Expr) extends Expr

/* References - Implicit */
case class LetVar(x: String, e1: Expr, e2: Expr) extends Expr
case class AssignVar(x: String, e1: Expr) extends Expr

/* References - Explicit */

case class Newref(e: Expr) extends Expr
case class Deref(e: Expr) extends Expr
case class Assignref(e1: Expr, e2: Expr) extends Expr 


defined trait Value
defined trait Environment
defined trait Expr
defined type Address
defined type RefCount
defined object ErrorValue
defined class NumValue
defined class BoolValue
defined class Closure
defined class Reference
defined object EmptyEnvironment
defined class Extend
defined class ExtendRec
defined function lookupEnv
defined class Const
defined class ConstBool
defined class Ident
defined class Plus
defined class Mult
defined class Geq
defined class IfThenElse
defined class Let
defined class Fundef
defined class Funcall
defined class LetRec
defined class LetVar
defined class AssignVar
defined class Newref
defined class Deref
defined class Assignref

In [21]:
type Store = Array[(Value, RefCount)]

def emptyStore : Store = Array[(Value, RefCount)]() 

def createNewCell(s: Store, v: Value): (Store, Address) = {
    val j = s.indexWhere(x => (x._2 == 0)) // Find first cell with 0 reference count
    if (j >= 0) { // if such a cell found
        println(s"From store: recycled cell # $j")
        s(j) = (v, 0) // allocate the value v to that cell, ref count is still 0
        (s, j) // return the new store and the address
    } else {
        val newAddr = s.length // we will create a new cell at the very end
        val s1 = s :+ (v, 0) // add a new cell at the end
        (s1, newAddr) // return the new store and the address at which we inserted our new elt.
    }
}

def lookup(s: Store, j: Address): Value = {
    val sValue = s(j) // Access the store at that address
    sValue._1
}

def assignCell(s: Store, j: Address, v: Value): Store = {
    val sValue = s(j) // Get the old contents
    val oldValue = sValue._1
    /*-- if the old value is a reference, decrement reference count --*/
    val s1 = oldValue match {
        case Reference(addr) => decrementReference(s, addr)
        case _ => s
    }
    s1(j) = (v, sValue._2) // Replace contents by value v but same ref count
    /*-- if the new value is a reference, increment reference count --*/
    val s2 = v match {
        case Reference(addr) => incrementReference(s1, addr)
        case _ => s1
    }
    s2
}

/*-- Special Reference Counting Operations --*/

def traverseChainAndOp(s: Store, addr: Address, 
                    visited: Set[Address], increment: Boolean): Store = {
    if (visited.contains(addr)){
        println("Cyclic pattern of references detected in the store.")
        s
    } else {
       val sValue = s(addr)
       s(addr) = {
           if (increment) {
             (sValue._1, sValue._2 + 1)
           } else {
              assert(sValue._2 >= 1)
              (sValue._1, sValue._2 - 1)
           }
       }
       sValue._1 match {
            case Reference(addr2) => {
                val newVisited = visited + addr
                traverseChainAndIncrement(s, addr2, newVisited)
            }
            case _ => s
        }
    }
}

def traverseChainAndIncrement(s: Store, addr: Address, visited: Set[Address]) = 
    traverseChainAndOp(s, addr, visited, true)

def traverseChainAndDecrement(s: Store, addr: Address, visited: Set[Address]) = 
    traverseChainAndOp(s, addr, visited, false)


def incrementReference(s: Store, j: Address): Store = {
    val sValue = s(j)
    assert(sValue._2 >= 0)
    s(j) = (sValue._1, sValue._2+1)
    sValue._1 match {
        case Reference(addr)  =>  traverseChainAndIncrement(s, addr, Set(j))
        case _ => s
    }
}

def decrementReference(s: Store, j: Address): Store = {
    val sValue = s(j)
    assert(sValue._2 >= 1)
    s(j) = (sValue._1, sValue._2-1)
    sValue._1 match {
        case Reference(addr) =>  traverseChainAndDecrement(s, addr, Set(j))
        case _ => s
    }   
}

defined type Store
defined function emptyStore
defined function createNewCell
defined function lookup
defined function assignCell
defined function traverseChainAndOp
defined function traverseChainAndIncrement
defined function traverseChainAndDecrement
defined function incrementReference
defined function decrementReference

## Garbage Collection using Reference Counting.

Let us first study the fragment of the language __without function calls/recursion__.  Once we handle this fragment, we will study the fragment with function calls, closures and such to complete the handling of garbage collection.

> A reference count is associated with each cell in the store that counts how many vars in the environment refer to a given cell. 

> In particular, if the reference count is zero for a cell, then there is no way to access that cell from the environment. It can be "garbage collected": i.e, its contents replaced on the next allocation.

Reference counting is a seamless approach to garbage collection. All it needs is a way for us to indicate at each cell how many references exist to that cell. How is this done? 
This is how: 

  - When a let binding or a let var binding happens, we note that a new identifier comes in scope. 
    - If the identifier that just came in scope is bound to a reference to a cell in the memory, __increment__ its reference count.
    - Note that incrementing the reference count of one cell may trigger reference count increases for other cells as well. How?
      - Imagine a cell address i whose contents are themselves a reference to another cell with address j.
      - It is important that if reference count of i goes up by 1, then so does that for cell j. Likewise, when we decrease reference counts.
  - When a let binding finishes evaluating, the identifier it binds goes out of scope.
    - Decrement its reference count if the identifier refers to a cell in the memory/if the cell refers to another cell, recursively decrease the ref count for that cell.
  - If the reference count for a cell is down to 0, it can be reused upon the next allocation of a new cell.
    
> Our solution works for both implicit and explicit references. For simplicity, we will demonstrate it for just the language with explicit references. But the language for implicit references works in a similar fashion: in fact it would be somewhat simpler for the implicit language.
    
### Example 4

Let us see how reference counting works.

```ocaml
let x = newref(10) in 
let y = newref( let z = newref(15) in  (* z comes into scope *)
                 let dummy = assignref(z, deref(x) - 5) in 
                    deref(z) + 20
               ) in (* z goes out of scope here *)
      deref(y)
```

How does the execution work with respect to the environment and store? We show below through a sequence of snapshots that include environment shown in yellow and the corresponding store shown as a table in blue. We show how the reference count becomes 0 when z goes out of scope and later, how the cell for y recycles the old cell that used to hold the pointer z.

<img src="figures/figExample4.png" width=65%>


In [22]:
def evalExpr(e: Expr, env: Environment, s: Store): (Value, Store) = {

def binopNumHelper(e1: Expr, e2: Expr) ( foo: (Double, Double) => Value ) = {
        val (v1, s1) = evalExpr(e1, env, s)
        v1 match {
            case NumValue(f1) => {
                val (v2, s2) = evalExpr(e2, env, s1)
                v2 match {
                    case NumValue(f2) => (foo(f1, f2), s2)
                    case _ => throw new IllegalArgumentException("Cannot op non numeric values")
                }
            }
            case _ => throw new IllegalArgumentException("Cannot op non numeric values")
        }
    }
    
    e match {
        case Const(f) => (NumValue(f), s)
        case ConstBool(b) => (BoolValue(b), s)
        case Ident(x) => { (lookupEnv(x, env), s) }
        case Plus(e1, e2) => binopNumHelper(e1, e2) ((f1, f2) => NumValue(f1 + f2))
        case Mult(e1, e2) => binopNumHelper(e1, e2) ((f1, f2) => NumValue(f1 * f2))
        case Geq(e1, e2) => binopNumHelper(e1, e2) ((f1, f2) => BoolValue(f1 >= f2))
        case IfThenElse(e1, e2, e3) => {
            val (v1, s1) = evalExpr(e1, env, s)
            v1 match {
                case BoolValue(true) => evalExpr(e2, env, s1)
                case BoolValue(false) => evalExpr(e3, env, s1)
                case _ => throw new IllegalArgumentException("If then else condition needs to be a boolean value")
            }
        }
        
        case Let(x, e1, e2) => {
            val (v1, s1) = evalExpr(e1, env, s)
            val s2: Store = /*-- lets add to reference count if needed --*/
                  v1 match {
                    case Reference(addr) => incrementReference(s1, addr)
                    case _ => s1
                   }
            val newEnv = Extend(x, v1, env)
            val (v2, s3) = evalExpr(e2, newEnv, s2)
            val s4: Store = /* -- lets decrement reference count if needed --*/
                v1 match {
                    case Reference(addr) => decrementReference(s1, addr)
                    case _ => s1
                   }
            (v2, s4)
        }
        
        case Newref(e1) => {
            val (v1, s1) = evalExpr(e1, env, s)
            val (s2, addr) = createNewCell(s1, v1)
            (Reference(addr), s2)
        }
        
        case Deref(e1) => {
            val (v1, s1) = evalExpr(e1, env, s)
            val v2 = v1 match {
                case Reference(addr) => lookup(s1, addr)
                case _ => throw new IllegalArgumentException("Deref a non reference value is not permitted")
            }
            (v2, s1)
        }
        
        case Assignref(e1, e2) => {
            val (v2, s1) = evalExpr(e2, env, s)
            val (v1, s2) = evalExpr(e1, env, s1)
            val s3 = v1 match {
                case Reference(addr) => assignCell(s2, addr, v2)   
                case _ => throw new IllegalArgumentException("Assignref to a non reference value is not permitted")
            }
            (v2, s3)
        }
        
        

    }
}

defined function evalExpr

In [23]:
/*
let x = newref(10) in 
let y = newref( let z = newref(15) in  (* z comes into scope *)
                 let dummy = assignref(z, deref(x) + 5) in 
                    deref(z) + 20
               ) in (* z goes out of scope here *)
      deref(y)
      */

val x = Ident("x")
val y = Ident("y")
val z = Ident("z")

val prog = Let("x", Newref(Const(10)), 
               Let("y", Newref( Let("z", Newref(Const(15)),
                                   Let("dummy", Assignref(z, Plus(Deref(x), Const(5))),
                                       Plus(Deref(z), Const(20))
                                      )
                                    )
                              ),
                   Deref(y)
                  ))

val v = evalExpr(prog, EmptyEnvironment, emptyStore)

From store: recycled cell # 1


x: Ident = Ident("x")
y: Ident = Ident("y")
z: Ident = Ident("z")
prog: Let = Let(
  "x",
  Newref(Const(10.0)),
  Let(
    "y",
    Newref(
      Let(
        "z",
        Newref(Const(15.0)),
        Let(
          "dummy",
          Assignref(Ident("z"), Plus(Deref(Ident("x")), Const(5.0))),
          Plus(Deref(Ident("z")), Const(20.0))
        )
      )
    ),
    Deref(Ident("y"))
  )
)
v: (Value, Store) = (NumValue(35.0), Array((NumValue(10.0), 0)))

### Adding Support For Function Calls.

When adding support for function calls, it is important to note that we may have closures whose environments may have captured a reference.

  - It is important when a closure is created to make sure that we account for references that are captured by the closure environment. 
  
Next, when we have a function call: `fun (arg)`:
  - Suppose `fun` is a `Closure(param, body, env)` then if `arg` is a reference to a cell in the memory then `param` is now also a reference to that cell.
    - In this case, we need to increment the reference count before starting to execute the body of the function.
    - When the function is done executing, we need to decrement the reference count.
    
Curried function calls pose an extra challenge.
  - There is a temporary closure that is created which may change the reference counts as we execute a curried function call.
  
> We will leave some of the details to you as an extra credit assignment if you would like to fully implement garbage collection for lettuce with function calls.


## Reference Counting: Advantages and Disadvantages

The main advantage of reference counting is that garbage collection is seamlessly integrated into the workings of the interpreter. As soon as a memory cell goes out of scope, we can reclaim memory and reuse it. In this sense, it is very memory efficient. 

There are many disadvantages to this scheme:
  - Every let binding, function call etc.. has a overhead which can be large if the program has created a complicated structure in the memory.
  - Imagine a binary tree implementation with reference counting:
     - Every time, we point to the root the garbage collector will automatically increment the reference count of every object. 
     - This can cause huge hidden performance overheads to implementations.
  - Another disadvantage is the extra overhead of storing the reference count for each object. This can easily become nontrivial.
  - Yet another disadvantage is when we create reference cycles: for instance in graph data structures or doubly linked list structures. These cycles can break reference counting.
  - A  final and very important disadvantage is when we support concurrent threads/processes. When multiple processes/threads share memory the reference counts have to be incremented/decremented atomically. In a multiprocessor system, this has a high overhead including forcing caches to synchronize across different processors. You have to use complicated constructs known as fences if your memory does not have sequential consistency. All of this raises the complexity of the implementation substantially.

## Alternatives to Reference Counting

There are alternatives which are more popular. You can look up the `Mark and Sweep` garbage collector algorithm that periodically traverses the reachable memory cells and reclaims memory when needed. This means that programs that do not consume much memory can run fast without garbage collection, which is invoked only when the program starts to run out of memory or the fragmentation of the allocated memory is high.

Many JVM implementations use mark and sweep algorithm variants for their garbage collection.  Python on the other hand uses reference counting (at least the CPython implementation does). But it does not handle circular references and it does not do reference counting for all types of objects.